In [15]:
using ForwardDiff

# Dubins car parallel park
n = 3 # number of states
m = 2 # number of controls

# initial and goal states
x0 = [0.;0.;0.]
xf = [0.;1.;0.]

# costs
Q = 0.001*eye(n)
Qf = 1000.0*eye(n)
R = 0.001*eye(m)

dt = 0.01

# RK4 integration
function fc(x,u)
    return [u[1]*cos(x[3]); u[1]*sin(x[3]); u[2]]
end

function rk4(f::Function,dt::Float64)
    # Runge-Kutta 4
    k1(x,u) = dt*f(x,u)
    k2(x,u) = dt*f(x + k1(x,u)/2.,u)
    k3(x,u) = dt*f(x + k2(x,u)/2.,u)
    k4(x,u) = dt*f(x + k3(x,u), u)
    fd(x,u) = x + (k1(x,u) + 2.*k2(x,u) + 2.*k3(x,u) + k4(x,u))/6.
end

function Jacobian(f::Function,x::Array{Float64,1},u::Array{Float64,1})
    f1 = a -> f(a,u)
    f2 = b -> f(x,b)
    fx = ForwardDiff.jacobian(f1,x)
    fu = ForwardDiff.jacobian(f2,u)
    return fx, fu
end

fd = rk4(fc, dt)
F(x,u) = Jacobian(fd,x,u)

X = [1.;1.;1.]
U = [1.;1.];

In [18]:
# Standard backward pass
println("Standard backward pass:")
@time begin 
    S = Qf
    s = Qf*(X - xf)

    lx = Q*(X - xf)
    lu = R*(U)
    lxx = Q
    luu = R
    fx, fu = F(X,U)
    Qx = lx + fx'*s
    Qu = lu + fu'*s
    Qxx = lxx + fx'*S*fx
    Quu = luu + fu'*S*fu
    Qux = fu'*S*fx
    K = Quu\Qux
    d = Quu\Qu
    s_ = Qx - K'*Qu + K'*Quu'*d - Qux'*d
    S_ = Qxx + K'*Quu*K - K'*Qux - Qux'*K
end
println("S=\n$S\nS_=\n$S_")
println("s=\n$s\ns_=\n$s_")
println("K=\n$K")
println("Quu=\n$Quu")
println("d=\n$d")

Standard backward pass:
  0.000253 seconds (262 allocations: 24.828 KiB)
S=
[1000.0 0.0 0.0; 0.0 1000.0 0.0; 0.0 0.0 1000.0]
S_=
[715.438 -448.053 -4.26243; -448.053 294.44 2.70697; -4.26243 2.70697 9.92749]
s=
[1000.0, 0.0, 1000.0]
s_=
[719.565, -450.791, 5.51555]
K=
[53.0783 83.5807 0.000816934; -0.417894 0.265376 99.0124]
Quu=
[0.100999 -8.33326e-7; -8.33326e-7 0.101002]
d=
[53.089, 98.5995]


In [17]:
## Square-root backward pass
println("Square-root backward pass")
@time begin 
    S = Qf
    s = Qf*(X - xf)
    A, B = F(X,U) 
    lx = Q*(X - xf)
    lu = R*U
    lxx = Q
    luu = R
    Qx = lx + fx'*s
    Qu = lu + fu'*s
    
    L = chol(S)
    Lq = chol(lxx)
    Lr = chol(luu)
    
    # K update
    K_sqrt = inv(luu)*B'*L*inv(eye(size(L,2)) + L'*B*inv(luu)*B'*L)*L'*A
    
    # d update
    H = [B'*L Lr]
    W0, V0 = qr(H)
    L_quu = W0*(V0*V0')^(1/2)
    
    d_sqrt = inv(L_quu*L_quu')*Qu
    
    # S_k+1 -> S_k update
    G = L'*B*inv(Lr')
    d1, W1 = eig(G*G')
    D1 = diagm(d1)
    P = [A'*L*W1*((eye(size(D1,1)) + D1)^(-1/2))*W1' Lq]
    W1, V = qr(P)
    d2, W2 = eig(V*V')
    D2 = diagm(d2)
    L_ = W1*W2*D2^(1/2)
    
    # s_k+1 -> s_k update
    s_sqrt = Qx - A'*L*inv(eye(size(L,2)) + L'*B*inv(luu)'*B'*L)*L'*B*inv(luu)'*Qu
    
end

#check results
println("S=\n$S")
println("L*L'=\n$(L*L')\n")
println("S_=\n$S_")
println("L_*L_'=\n$(L_*L_')\n")
println("s_=\n$s_")
println("s_qrt=\n$s_sqrt\n")
println("K=\n$K")
println("K_sqrt=\n$K_sqrt\n")
println("Quu=\n$Quu")
println("L_quu*L_quu'=\n$(L_quu*L_quu')\n")
println("d:\n$d")
println("d_sqrt=\n$d_sqrt\n")

Square-root backward pass
  0.000581 seconds (522 allocations: 54.250 KiB)
S=
[1000.0 0.0 0.0; 0.0 1000.0 0.0; 0.0 0.0 1000.0]
L*L'=
[1000.0 0.0 0.0; 0.0 1000.0 0.0; 0.0 0.0 1000.0]

S_=
[715.438 -448.053 -4.26243; -448.053 294.44 2.70697; -4.26243 2.70697 9.92749]
L_*L_'=
[715.438 -448.053 -4.26243; -448.053 294.44 2.70697; -4.26243 2.70697 9.92749]

s_=
[719.565, -450.791, 5.51555]
s_qrt=
[719.565, -450.791, 5.51555]

K=
[53.0783 83.5807 0.000816934; -0.417894 0.265376 99.0124]
K_sqrt=
[53.0783 83.5807 0.000816934; -0.417894 0.265376 99.0124]

Quu=
[0.100999 -8.33326e-7; -8.33326e-7 0.101002]
L_quu*L_quu'=
[0.100999 -8.33326e-7; -8.33326e-7 0.101002]

d:
[53.089, 98.5995]
d_sqrt=
[53.089, 98.5995]

